In [1]:
# -*- coding: utf8 -*-
import sys
sys.path.insert(1,'../')
#tried to do the whole "..package_dir.module" thing but kept getting a relative import error I
#could not fix by putting '__init___.py' files everywhere

In [2]:
import os
import json
from astropy.io import fits
from astropy import units
import glob
import pandas as pd
import numpy as np
import ccdproc
from ccdproc import CCDData
import warnings
warnings.filterwarnings('ignore')
import logging
import argparse
import logging
from importlib import reload
from SAMOS_DRP.SAMOS_NIGHT import SAMOSNight
from SAMOS_DRP.ImageProcessor import ImageProcessor
from SAMOS_DRP.SAMOSHelpers import load_bucket_status,save_bucket_status
from SAMOS_DRP.SlitBuckets import SlitBuckets
from SAMOS_DRP.Spectroscopy.wavelength import WavelengthCalibration
from SAMOS_DRP.DoWavecal import WaveCalBuckets

# INITIALIZATION


Step 1: organize the data and stores relevant information based on FITS headers.


Observation information is organized into a pandas dataframe and further divided into `buckets`.
Main buckets are: 
<h3> Bias frames:</h3> not currently implemented, bias is accounted for using an overscan region.
<h3>Flats:</h3> quartz flats to characterize nonuniformity in CCD pixel sensitivity.
<h3>Comparison lamps:</h3> Arc lamp exposures for wavelength calibration.
<h3>Science/Comparison pairs:</h3> Object exposures and the corresponding comparison lamps.
    
<h3>Initialization setup:</h3> 

`SAMOSNight(obsid=SAMOS_obsid,
            raw_data_dir=raw_data_dir,
            proc_dir=proc_dir,
            ignore_bias=True)`



#### Params:
`obsid` the observation ID corresponding to the SAMOS data you want to reduce (not necessary currently, but will be implemented in the future) \
`raw_data_dir` directory containing the raw observation data (for testing purposes) \
`proc_dir` directory for storage of pipeline results \
`ignore_bias` whether or not to ignore bias frames for correction (set to true because of overscan region in test data)

In [3]:
LOG_FILENAME = 'testing_DRP.log'
if os.path.exists(LOG_FILENAME):
    os.remove(LOG_FILENAME)

raw_data_dir = os.path.join(os.path.abspath('../'),'UNCOMP_GDMN_DATA')
    
logging.basicConfig(filename=LOG_FILENAME,level=logging.DEBUG)
GDMN_Night1 = SAMOSNight(obsid='QUARANTINE',
                         raw_data_dir=raw_data_dir,
                         proc_dir=os.path.join(os.getcwd(),'testing'),
                         ignore_bias=True) #setup
GDMN_Night1() #call

/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/UNCOMP_GDMN_DATA
True
{'obsid': 'QUARANTINE', 'data_path': '/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/UNCOMP_GDMN_DATA', 'gain': 2.06, 'rdnoise': 3.99, 'ccdsum': '1 1', 'empty_bucket': True, 'bias_buckets': None, 'flat_buckets': None, 'comp_buckets': None, 'targ_buckets': None, 'spec_buckets': None, 'slit_buckets': None, 'fits_slit_locs': None, 'sunset_time': None, 'sunrise_time': None, 'evening_twilight': None, 'morning_twilight': None, 'header_keys': ['naxis', 'date', 'slit', 'date-obs', 'obstype', 'object', 'exptime', 'obsra', 'obsdec', 'grating', 'cam_targ', 'grt_targ', 'filter', 'filter2', 'gain', 'rdnoise', 'ccdsum', 'wavmode']}
added flat bucket                          file  naxis        date                 slit  \
0  psg_140319_210352_fri.fits      2  2014-03-19  Ori0B1-field2-0000R   
1  psg_140319_210620_fri.fits      2  2014-03-19  Ori0B1-field2-0000R   
2  psg_140319_210744_fri

[<class 'SAMOS_DRP.DataBucket.DataBucket'>
 SAMOS Observation ID: QUARANTINE
 Data Directory: /Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/UNCOMP_GDMN_DATA
 Gain: 2.06
 Readout Noise: 3.99
 ccdsum: 1 1
 Is Empty: False
 
 Bucket Grouping Information
 
 (small buckets inside bigger bucekts!)
 BIAS Group:
   Group is Empty
 FLATs Group:
 Files in BUCKET
   psg_140319_210352_fri.fits
   psg_140319_210620_fri.fits
   psg_140319_210744_fri.fits
   psg_140319_210907_fri.fits
   psg_140319_211031_fri.fits
   psg_140319_211154_fri.fits
   psg_140319_211318_fri.fits
   psg_140319_211442_fri.fits
   psg_140319_211605_fri.fits
   psg_140319_211729_fri.fits
 COMP Group:
 Files in BUCKET
   psg_140320_005007_cri.fits
   psg_140320_005319_cri.fits
   psg_140320_005443_cri.fits
   psg_140320_005607_cri.fits
   psg_140320_005731_cri.fits
   psg_140320_005854_cri.fits
   psg_140320_010018_cri.fits
   psg_140320_010142_cri.fits
   psg_140320_010305_cri.fits
   psg_140320_010429_c

In [4]:

GDMN_N1_ccproc = ImageProcessor(GDMN_Night1)
GDMN_N1_ccproc()


/Volumes/DanaExternalMacbook/PhD_Thesis/SAMOS_Project/SAMOS_pipeline/UNCOMP_GDMN_DATA
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS f

In [5]:
#reload(SAMOS_DRP.SlitBuckets)
from SAMOS_DRP.SlitBuckets import SlitBuckets
GDMN_N1_slits = SlitBuckets(GDMN_N1_ccproc)
GDMN_N1_slits()

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
number of cols: 4060.000000
number of rows: 1896.000000
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
number of cols: 4060.000000
number of rows: 1896.000000


In [ ]:
#reload(SAMOS_DRP.DoWavecal)


GDMN_N1_wavecals = WaveCalBuckets(GDMN_N1_slits)

GDMN_N1_wavecals()